# Homework 4

## FINM 37400 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# 1 HBS Case: The Information Content of the Yield Curve

## 1.1

The Term Spread
* What is the term spread? 
* What is the preferred measure of the term spread in the case?
* What evidence is cited for the term spread's power in predicting recessions?
* Which is more important, the level of the term spread, its month-over-month change, or the level of short rates?

Using the file `yields_2022-11-30.xlsx`, calculate the term spread as of the end of November 2022. (Measure it with the 10-year and 1-year yields.)
* What does this term spread suggest about economic growth, according to the evidence of Estrella?

## 1.2

Term premia
* What is the term premium? Why is it impossible to get a direct measure of it?
* Explain equations (1) and (2). Why does Sacks think this concept of the term spread complicates the relationship found by Estrella?

## 1.3

Though this case touches on monetary policy, that is not our focus. Rather, our focus is the degree to which the current shape of the yield curve is predictive of future short-term interest rates.

Suppose the term premium is zero. 
* What does an upward sloping yield curve predict about future short-term rates? 
* In this case, is it risky to hold long-term bonds? Is this risk compensated?

Suppose the term premium increases 3 percentage points.
* What would change about your previous answer?

# 2 Calculating Forwards

Use the treasury issues in `'../data/selected_treasury_rates.xlsx.xlsx`. 

The selected treasuries are at nearly six-month intervals. Approximate that they mature at perfect 6-month intervals. (We are looking at them from the perspective of March 31, 2022.)

## 2.1 
Bootstrap the discount factor. Plot the answer in terms of discount factors and discount rates.

## 2.2
Calculate the entire forward curve for rates from $T$ to $T+0.5$, ranging $T$ from 0.5 through 6.5.

Plot the forward discount curve.

## 2.3
Use the discount factors to calculate the forward rate from $T_1=2$ to $T_2=2.5$.

Do the same for $T_1=5$ to $T_2=5.5$.

## 2.4
Explain how an FRA between $T_1$ and $T_2$ would function. How is the rate calculated above involved?



In [10]:
import pandas as pd
import numpy as np
import datetime
import warnings
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
import scipy.optimize as optimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13
import seaborn as sns
sns.set(rc={'figure.figsize':(15, 10)})
import treasury_cmds


In [8]:
rates = pd.read_excel("C:/Users/dcste/OneDrive/fixed_income/fixed_income_FORKED/finm-fixedincome-2023/data/selected_treasury_rates.xlsx")
rates.head()

,KYTREASNO,issue date,maturity date,maturity,coupon rate,price,ytm
0,206896,2015-09-30,2022-09-30,0.501027,1.750,100.363281,0.010144
1,207688,2021-03-31,2023-03-31,0.999316,0.125,98.457031,0.016804
2,207775,2021-09-30,2023-09-30,1.500342,0.250,97.218750,0.021307
3,207077,2017-03-31,2024-03-31,2.001369,2.125,99.570312,0.023294
4,207416,2019-09-30,2024-09-30,2.502396,1.500,97.621094,0.024680


In [16]:
df1 =rates.set_index("KYTREASNO", )
df1

,issue date,maturity date,maturity,coupon rate,price,ytm
KYTREASNO,,,,,,
206896,2015-09-30,2022-09-30,0.501027,1.750,100.363281,0.010144
207688,2021-03-31,2023-03-31,0.999316,0.125,98.457031,0.016804
207775,2021-09-30,2023-09-30,1.500342,0.250,97.218750,0.021307
207077,2017-03-31,2024-03-31,2.001369,2.125,99.570312,0.023294
207416,2019-09-30,2024-09-30,2.502396,1.500,97.621094,0.024680
207507,2020-03-31,2025-03-31,3.000684,0.500,94.175781,0.025101
207256,2018-09-30,2025-09-30,3.501711,3.000,101.554688,0.025142
207337,2019-03-31,2026-03-31,4.000000,2.250,98.929688,0.025154
207777,2021-09-30,2026-09-30,4.501027,0.875,93.042969,0.025020


In [18]:
df1 = df1.rename(columns={'maturity date': 'TMATDT','issue date':'CALDT','coupon rate':'TCOUPRT'})

In [20]:
treasury_cmds.calc_cashflows(df1)

,2016-03-30,2016-09-30,2017-03-30,2017-09-30,2018-03-30,2018-09-30,2019-03-30,2019-09-30,2020-03-30,2020-09-30,...,2026-03-31,2026-09-30,2027-03-30,2027-03-31,2027-09-30,2028-03-30,2028-03-31,2028-09-30,2029-03-30,2029-03-31
KYTREASNO,,,,,,,,,,,,,,,,,,,,,
206896,0.875,0.875,0.875,0.8750,0.8750,0.8750,0.8750,0.8750,0.8750,0.8750,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207688,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207775,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207077,0.000,0.000,0.000,1.0625,1.0625,1.0625,1.0625,1.0625,1.0625,1.0625,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207416,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7500,0.7500,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207507,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2500,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207256,0.000,0.000,0.000,0.0000,0.0000,0.0000,1.5000,1.5000,1.5000,1.5000,...,0,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207337,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,1.1250,1.1250,1.1250,...,100,0.0000,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
207777,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,100.4375,0.0000,0,0.0000,0.0000,0,0.0000,0.0000,0
